In [1]:
from elasticsearch import Elasticsearch
# from datasets import load_from_disk

In [2]:
es = Elasticsearch('http://localhost:9200')

In [3]:
es.info()

/var/folders/lp/tb5q9ks973gdt01t1w07s95m0000gn/T/ipykernel_8779/515033876.py:1: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.info()


ObjectApiResponse({'name': '82365160d178', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'RSL0QICXQ6uGZJ-EP_X6mA', 'version': {'number': '7.17.4', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '79878662c54c886ae89206c685d9f1051a9d6411', 'build_date': '2022-05-18T18:04:20.964345128Z', 'build_snapshot': False, 'lucene_version': '8.11.1', 'minimum_wire_compatibility_version': '6.8.0', 'minimum_index_compatibility_version': '6.0.0-beta1'}, 'tagline': 'You Know, for Search'})

In [5]:
!curl -XGET localhost:9200

{
  "name" : "82365160d178",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "RSL0QICXQ6uGZJ-EP_X6mA",
  "version" : {
    "number" : "7.17.4",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "79878662c54c886ae89206c685d9f1051a9d6411",
    "build_date" : "2022-05-18T18:04:20.964345128Z",
    "build_snapshot" : false,
    "lucene_version" : "8.11.1",
    "minimum_wire_compatibility_version" : "6.8.0",
    "minimum_index_compatibility_version" : "6.0.0-beta1"
  },
  "tagline" : "You Know, for Search"
}


In [45]:
INDEX = 'search_test'
es.indices.create(index=INDEX)

/var/folders/lp/tb5q9ks973gdt01t1w07s95m0000gn/T/ipykernel_8779/515627836.py:2: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.indices.create(index=INDEX)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'search_test'})

In [46]:
!curl -XGET localhost:9200/_cat/indices?v

zsh:1: no matches found: localhost:9200/_cat/indices?v


In [9]:
import json
with open("anno.json", "r", encoding="utf-8") as json_file:
    latex_data = json.load(json_file)

In [36]:
latex_data['latex_anno'][0]

{'latex': ['u,v\\in \\Gamma (\\mathrm {T} M)',
  '\\operatorname {Riem} (u,v)w=\\nabla _{u}\\nabla _{v}w-\\nabla _{v}\\nabla _{u}w-\\nabla _{[u,v]}w',
  '{\\operatorname {Riem} ^{\\rho }}_{\\sigma \\mu \\nu }=\\partial _{\\mu }\\Gamma _{\\nu \\sigma }^{\\rho }-\\partial _{\\nu }\\Gamma _{\\mu \\sigma }^{\\rho }+\\Gamma _{\\mu \\lambda }^{\\rho }\\Gamma _{\\nu \\sigma }^{\\lambda }-\\Gamma _{\\nu \\lambda }^{\\rho }\\Gamma _{\\mu \\sigma }^{\\lambda }',
  '\\operatorname {Riem} (u,v)=-\\operatorname {Riem} (v,u)',
  '\\langle \\operatorname {Riem} (u,v)w,z\\rangle =-\\langle \\operatorname {Riem} (u,v)z,w\\rangle ',
  '\\langle \\operatorname {Riem} (u,v)w,z\\rangle =\\langle \\operatorname {Riem} (w,z)u,v\\rangle ',
  '\\operatorname {Riem} (u,v)w+\\operatorname {Riem} (v,w)u+\\operatorname {Riem} (w,u)v=0',
  '(\\nabla _{u}\\operatorname {Riem} )(v,w)+(\\nabla _{v}\\operatorname {Riem} )(w,u)+(\\nabla _{w}\\operatorname {Riem} )(u,v)=0',
  '\\operatorname {Riem} _{(\\rho \\sigma )\\mu

In [41]:
contexts = list()
for v in latex_data['latex_anno']:
    id, link, title = v['id'], v['link'], v['title']
    for latex in v['latex']:
        contexts.append((latex, id, link, title))

print(f"Lengths of unique contexts : {len(contexts)}")

Lengths of unique contexts : 289


In [42]:
print(contexts[0])
print(contexts[1])

('u,v\\in \\Gamma (\\mathrm {T} M)', 'dc9fd2c45824afc', 'https://ko.wikipedia.org/wiki/%EB%A6%AC%EB%A7%8C_%EA%B3%A1%EB%A5%A0', '리만 곡률 텐서')
('\\operatorname {Riem} (u,v)w=\\nabla _{u}\\nabla _{v}w-\\nabla _{v}\\nabla _{u}w-\\nabla _{[u,v]}w', 'dc9fd2c45824afc', 'https://ko.wikipedia.org/wiki/%EB%A6%AC%EB%A7%8C_%EA%B3%A1%EB%A5%A0', '리만 곡률 텐서')


In [ ]:
# contexts = list(
#     dict.fromkeys([latex for latex in v["latex"] for v in latex_data["latex_anno"]])
# )  # set 은 매번 순서가 바뀌므로
# print(f"Lengths of unique contexts : {len(contexts)}")

In [52]:
for idx, data in enumerate(contexts):
    latex, id, link, title = data
    body = {'latex': latex, 'id': id, 'link': link, 'title': title}
    print(body)
    es.index(index=INDEX, id=idx+1, body=body)

{'latex': 'u,v\\in \\Gamma (\\mathrm {T} M)', 'id': 'dc9fd2c45824afc', 'link': 'https://ko.wikipedia.org/wiki/%EB%A6%AC%EB%A7%8C_%EA%B3%A1%EB%A5%A0', 'title': '리만 곡률 텐서'}


/var/folders/lp/tb5q9ks973gdt01t1w07s95m0000gn/T/ipykernel_8779/3789976582.py:5: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.index(index=INDEX, id=idx+1, body=body)
/var/folders/lp/tb5q9ks973gdt01t1w07s95m0000gn/T/ipykernel_8779/3789976582.py:5: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.index(index=INDEX, id=idx+1, body=body)


{'latex': '\\operatorname {Riem} (u,v)w=\\nabla _{u}\\nabla _{v}w-\\nabla _{v}\\nabla _{u}w-\\nabla _{[u,v]}w', 'id': 'dc9fd2c45824afc', 'link': 'https://ko.wikipedia.org/wiki/%EB%A6%AC%EB%A7%8C_%EA%B3%A1%EB%A5%A0', 'title': '리만 곡률 텐서'}
{'latex': '{\\operatorname {Riem} ^{\\rho }}_{\\sigma \\mu \\nu }=\\partial _{\\mu }\\Gamma _{\\nu \\sigma }^{\\rho }-\\partial _{\\nu }\\Gamma _{\\mu \\sigma }^{\\rho }+\\Gamma _{\\mu \\lambda }^{\\rho }\\Gamma _{\\nu \\sigma }^{\\lambda }-\\Gamma _{\\nu \\lambda }^{\\rho }\\Gamma _{\\mu \\sigma }^{\\lambda }', 'id': 'dc9fd2c45824afc', 'link': 'https://ko.wikipedia.org/wiki/%EB%A6%AC%EB%A7%8C_%EA%B3%A1%EB%A5%A0', 'title': '리만 곡률 텐서'}
{'latex': '\\operatorname {Riem} (u,v)=-\\operatorname {Riem} (v,u)', 'id': 'dc9fd2c45824afc', 'link': 'https://ko.wikipedia.org/wiki/%EB%A6%AC%EB%A7%8C_%EA%B3%A1%EB%A5%A0', 'title': '리만 곡률 텐서'}
{'latex': '\\langle \\operatorname {Riem} (u,v)w,z\\rangle =-\\langle \\operatorname {Riem} (u,v)z,w\\rangle ', 'id': 'dc9fd2c458

In [53]:
es.get(index=INDEX, id=250)

/var/folders/lp/tb5q9ks973gdt01t1w07s95m0000gn/T/ipykernel_8779/3179673133.py:1: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.get(index=INDEX, id=250)


ObjectApiResponse({'_index': 'search_test', '_type': '_doc', '_id': '250', '_version': 1, '_seq_no': 249, '_primary_term': 1, 'found': True, '_source': {'latex': '\\therefore \\beta ={{\\ln 10} \\over {2L}}', 'id': '88e49440a52b8bd', 'link': 'https://ko.wikipedia.org/wiki/%EB%A0%88%EB%B9%84_%EC%83%81%EC%88%98', 'title': '레비 상수'}})

In [54]:
body = {
		'query': {
				'match': {
						'latex': '\\therefore \\beta ={{\\ln 50} \\over {32L}}'
				}
		}
}
res = es.search(index=INDEX, body=body)
print(res)

/var/folders/lp/tb5q9ks973gdt01t1w07s95m0000gn/T/ipykernel_8779/1388031282.py:8: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  res = es.search(index=INDEX, body=body)


{'took': 804, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 34, 'relation': 'eq'}, 'max_score': 16.99393, 'hits': [{'_index': 'search_test', '_type': '_doc', '_id': '250', '_score': 16.99393, '_source': {'latex': '\\therefore \\beta ={{\\ln 10} \\over {2L}}', 'id': '88e49440a52b8bd', 'link': 'https://ko.wikipedia.org/wiki/%EB%A0%88%EB%B9%84_%EC%83%81%EC%88%98', 'title': '레비 상수'}}, {'_index': 'search_test', '_type': '_doc', '_id': '249', '_score': 11.381777, '_source': {'latex': '\\;\\;\\;={{\\ln 10} \\over {2\\beta }}', 'id': '88e49440a52b8bd', 'link': 'https://ko.wikipedia.org/wiki/%EB%A0%88%EB%B9%84_%EC%83%81%EC%88%98', 'title': '레비 상수'}}, {'_index': 'search_test', '_type': '_doc', '_id': '242', '_score': 10.282867, '_source': {'latex': '\\beta ={{\\pi ^{2}} \\over {12\\ln(2)}}', 'id': '88e49440a52b8bd', 'link': 'https://ko.wikipedia.org/wiki/%EB%A0%88%EB%B9%84_%EC%83%81%EC%88%98', 'title': '레비 상수'}}, {'_index': '

/var/folders/lp/tb5q9ks973gdt01t1w07s95m0000gn/T/ipykernel_8779/1388031282.py:8: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  res = es.search(index=INDEX, body=body)
